<a href="https://colab.research.google.com/github/Shurui-Zhang/Deep_learning/blob/main/Lab4_4_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 4: Using GPU acceleration with PyTorch

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 16.2MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [ ]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 2.1793,  0.4631,  1.6673,  ...,  0.1253,  0.8778,  0.2908],
        [ 1.1839,  0.4800,  2.3504,  ...,  0.1272, -0.1306, -0.0107],
        [ 0.1647, -0.2300,  0.1488,  ...,  0.2744, -0.0215,  0.9348],
        ...,
        [ 1.1598,  0.2586, -0.5674,  ...,  0.0081, -0.1844,  0.9132],
        [ 0.0875,  0.6965,  0.4186,  ...,  1.4514, -0.8568, -0.6353],
        [-0.8080,  1.2109,  0.1964,  ..., -0.5274, -0.8919, -0.1481]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [ ]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-5.6564e-01,  1.1213e-01,  1.3109e-01,  ..., -8.3360e-01,
         -1.2570e-01,  2.8083e-01],
        [ 3.0686e-01,  1.3409e+00, -2.5276e-02,  ..., -1.3434e+00,
          1.3546e+00, -3.3875e-01],
        [ 8.5285e-01, -1.7738e-01, -2.1186e-01,  ...,  2.8743e-01,
         -3.1022e-01,  8.7182e-01],
        ...,
        [ 2.8580e+00, -8.3393e-01, -2.8365e-02,  ..., -5.7143e-02,
         -6.2720e-02,  2.7064e-01],
        [ 1.2811e-01,  1.1178e+00, -1.1375e+00,  ..., -5.8960e-03,
          4.1079e-04,  2.4727e-01],
        [-4.2290e-02,  6.4153e-01,  2.1125e-01,  ...,  6.4105e-01,
          1.6455e-01, -1.6086e+00]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-4.2921e-02, -7.1025e-01,  7.7842e-02,  ...,  4.8752e+00,
          1.2914e+00, -8.0664e-01],
        [ 4.1644e-01,  1.3627e-02,  2.2329e-02,  ...,  4.1281e-01,
          2.6031e-01,  1.3871e+00],
        [ 8.9189e-01,  7.9713e-01, -8.1570e-02,  ...,  4.1516e-02,
         -3.4041e-01, -2.3625e-02],
        ...,
        [ 5.9846e-01,  1.0114e-02, -2.2416e-01,  ..., -1.5512e-01,
         -1.3778e-01,  1.7611e-01],
        [ 2.5308e-03, -8.9775e-01,  7.7542e-01,  ...,  6.4926e-02,
          6.2461e-01, -2.2996e+00],
        [ 1.0158e-01, -2.4610e+00, -9.0247e-01,  ..., -8.0667e-01,
         -2.6545e-01,  1.1912e+00]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device) #将modle拷贝到GPU上

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) #将数据拷贝到GPU上

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device) #告知trial在GPU上运行
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

YOUR ANSWER HERE

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

YOUR ANSWER HERE